# Pipeline Project

You will be using the provided data to create a machine learning model pipeline.

You must handle the data appropriately in your pipeline to predict whether an
item is recommended by a customer based on their review.
Note the data includes numerical, categorical, and text data.

You should ensure you properly train and evaluate your model.

## The Data

The dataset has been anonymized and cleaned of missing values.

There are 8 features for to use to predict whether a customer recommends or does
not recommend a product.
The `Recommended IND` column gives whether a customer recommends the product
where `1` is recommended and a `0` is not recommended.
This is your model's target/

The features can be summarized as the following:

- **Clothing ID**: Integer Categorical variable that refers to the specific piece being reviewed.
- **Age**: Positive Integer variable of the reviewers age.
- **Title**: String variable for the title of the review.
- **Review Text**: String variable for the review body.
- **Positive Feedback Count**: Positive Integer documenting the number of other customers who found this review positive.
- **Division Name**: Categorical name of the product high level division.
- **Department Name**: Categorical name of the product department name.
- **Class Name**: Categorical name of the product class name.

The target:
- **Recommended IND**: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.

## Load Data

In [1]:
import pandas as pd

# Load data
df = pd.read_csv(
    'data/reviews.csv',
)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18442 entries, 0 to 18441
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              18442 non-null  int64 
 1   Age                      18442 non-null  int64 
 2   Title                    18442 non-null  object
 3   Review Text              18442 non-null  object
 4   Positive Feedback Count  18442 non-null  int64 
 5   Division Name            18442 non-null  object
 6   Department Name          18442 non-null  object
 7   Class Name               18442 non-null  object
 8   Recommended IND          18442 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.3+ MB


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name,Recommended IND
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants,1
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses,1
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses,0
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits,1


In [2]:
df['Full Review'] = df['Title'].fillna('') + " " + df['Review Text'].fillna('')

## Preparing features (`X`) & target (`y`)

In [3]:
data = df

# separate features from labels
X = data.drop(['Title', 'Review Text', 'Recommended IND'], axis=1)
y = data['Recommended IND'].copy()

print('Labels:', y.unique())
print('Features:')
display(X.head())

Labels: [0 1]
Features:


,Clothing ID,Age,Positive Feedback Count,Division Name,Department Name,Class Name,Full Review
0,1077,60,0,General,Dresses,Dresses,Some major design flaws I had such high hopes ...
1,1049,50,0,General Petite,Bottoms,Pants,"My favorite buy! I love, love, love this jumps..."
2,847,47,6,General,Tops,Blouses,Flattering shirt This shirt is very flattering...
3,1080,49,4,General,Dresses,Dresses,Not for the very petite I love tracy reese dre...
4,858,39,1,General Petite,Tops,Knits,Cagrcoal shimmer fun I aded this in my basket ...


In [4]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    shuffle=True,
    random_state=27,
)

## Data Exploration

In [5]:
numeric_features = X.select_dtypes(exclude=['object']).columns
print("Numerical features:", numeric_features)

categorical_features = (X.select_dtypes(exclude=['int64']).columns
    .drop(['Full Review']))

print("Categorical features:", categorical_features)

text_features = (X[['Full Review']]).columns
print("Text features:", text_features)

Numerical features: Index(['Clothing ID', 'Age', 'Positive Feedback Count'], dtype='object')
Categorical features: Index(['Division Name', 'Department Name', 'Class Name'], dtype='object')
Text features: Index(['Full Review'], dtype='object')


## Building Pipeline

In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")

def spacy_token(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ('scaler', MinMaxScaler())
])
print("Numeric pipelines:\n", numeric_pipeline)

categorical_pipeline = Pipeline([
    ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ('cat_enocder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
print("Categorical pipelines:\n", categorical_pipeline)

text_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda X: X.iloc[:, 0], validate=False)),
    ('tfidf', TfidfVectorizer())
])
print("Text pipelines:\n", text_pipeline)


Numeric pipelines:
 Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler())])
Categorical pipelines:
 Pipeline(steps=[('ordinal_encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('cat_enocder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
Text pipelines:
 Pipeline(steps=[('selector',
                 FunctionTransformer(func=<function <lambda> at 0x00000271761ED580>)),
                ('tfidf', TfidfVectorizer())])


In [8]:
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer([
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features),
        ('text', text_pipeline, text_features)
])

preprocessing

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', MinMaxScaler())]),
                                 Index(['Clothing ID', 'Age', 'Positive Feedback Count'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('ordinal_encoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1)),
                                                 ('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_enocder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Division Name', 'Department Name', 'Class Name'], dtype='object')),
                                ('text',
                                 Pipeline(steps=[('selector',
                                                  FunctionTransformer(func=<function <lambda> at 0x00000271761ED580>)),
                                                 ('tfidf', TfidfVectorizer())]),
                                 Index(['Full Review'], dtype='object'))])

## Training Pipeline

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

dtc_model_pipeline = make_pipeline(preprocessing, DecisionTreeClassifier())
dtc_model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Clothing ID', 'Age', 'Positive Feedback Count'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1)),
                                                                  ('imputer',
                                                                   SimpleImpute...
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['Division Name', 'Department Name', 'Class Name'], dtype='object')),
                                                 ('text',
                                                  Pipeline(steps=[('selector',
                                                                   FunctionTransformer(func=<function <lambda> at 0x00000271761ED580>)),
                                                                  ('tfidf',
                                                                   TfidfVectorizer())]),
                                                  Index(['Full Review'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

## Fine-Tuning Pipeline

In [10]:
dtc_model_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'decisiontreeclassifier', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__num', 'columntransformer__cat', 'columntransformer__text', 'columntransformer__num__memory', 'columntransformer__num__steps', 'columntransformer__num__verbose', 'columntransformer__num__imputer', 'columntransformer__num__scaler', 'columntransformer__num__imputer__add_indicator', 'columntransformer__num__imputer__copy', 'columntransformer__num__imputer__fill_value', 'columntransformer__num__imputer__keep_empty_features', 'columntransformer__num__imputer__missing_values', 'columntransformer__num__imputer__strategy', 'columntransformer__num__scaler__clip', 'columntransformer__num__scaler__copy', 'columntransformer__num__scaler__feat

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'decisiontreeclassifier__max_depth':    [5, 10, 15],
    'decisiontreeclassifier__max_features': [150, 200, "sqrt"],
    'decisiontreeclassifier__splitter':     ["best", "random"],
    'decisiontreeclassifier__random_state': [1, 10, 100]
}

grid_search = GridSearchCV(dtc_model_pipeline, param_grid, cv=5, verbose=3)
grid_search.fit(X_train, y_train)

print("\nBest Hyperparameters:")
print(grid_search.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5] END decisiontreeclassifier__max_depth=5, decisiontreeclassifier__max_features=150, decisiontreeclassifier__random_state=1, decisiontreeclassifier__splitter=best;, score=0.816 total time=   2.0s
[CV 2/5] END decisiontreeclassifier__max_depth=5, decisiontreeclassifier__max_features=150, decisiontreeclassifier__random_state=1, decisiontreeclassifier__splitter=best;, score=0.817 total time=   2.3s
[CV 3/5] END decisiontreeclassifier__max_depth=5, decisiontreeclassifier__max_features=150, decisiontreeclassifier__random_state=1, decisiontreeclassifier__splitter=best;, score=0.822 total time=   2.1s
[CV 4/5] END decisiontreeclassifier__max_depth=5, decisiontreeclassifier__max_features=150, decisiontreeclassifier__random_state=1, decisiontreeclassifier__splitter=best;, score=0.816 total time=   2.1s
[CV 5/5] END decisiontreeclassifier__max_depth=5, decisiontreeclassifier__max_features=150, decisiontreeclassifier__random_sta

## Evaluation

In [12]:
from sklearn.metrics import accuracy_score

best_model = grid_search.best_estimator_

y_pred_best = best_model.predict(X_test)
accuracy_pipeline = accuracy_score(y_test, y_pred_best)

print("Accuracy:", accuracy_pipeline)

Accuracy: 0.8238482384823849
